# Lab 06: AgentCore Gateway

## Overview

In this notebook, we integrate **AgentCore Gateway** to centralize tools as MCP-compatible endpoints. Instead of embedding tools directly in each agent, we expose them through a Gateway that multiple agents can share.

**What you'll learn:**
- How to create an AgentCore Gateway with JWT authentication
- How to add Lambda functions as Gateway targets
- How to connect agents to Gateway via MCP client
- Token savings from centralized tool management

**Why Gateway Matters:**
- **Reusability**: Same tools serve multiple agents (support, sales, returns)
- **Security**: Centralized authentication and access control
- **Maintenance**: Update tools in one place, all agents benefit

## Prerequisites

- Completed Labs 01-05
- Infrastructure deployed (`make deploy-all`)

## Workshop Journey

```
01 Baseline → 02 Quick Wins → 03 Caching → 04 Routing → 05 Guardrails → [06 Gateway] → 07 Evaluations
                                                                             ↑
                                                                        You are here
```

## Step 1: Setup and Imports

In [ ]:
import os
import json
import uuid
import base64
import requests
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(override=True)

import boto3
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from bedrock_agentcore_starter_toolkit import Runtime

region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION", "us-east-1"))
ssm_client = boto3.client("ssm", region_name=region)
gateway_client = boto3.client("bedrock-agentcore-control", region_name=region)
cognito_client = boto3.client("cognito-idp", region_name=region)
agentcore_runtime = Runtime()

print(f"Region: {region}")
print(f"Langfuse Host: {os.environ.get('LANGFUSE_HOST', 'https://cloud.langfuse.com')}")

## Step 2: Helper Functions

In [ ]:
def get_ssm_parameter(name):
    """Retrieve a parameter from SSM Parameter Store."""
    response = ssm_client.get_parameter(Name=name)
    return response["Parameter"]["Value"]


def put_ssm_parameter(name, value):
    """Store a parameter in SSM Parameter Store."""
    ssm_client.put_parameter(Name=name, Value=value, Type="String", Overwrite=True)


def get_cognito_token(client_id, client_secret, token_url, scope):
    """Get OAuth2 token using client_credentials flow."""
    auth = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
    response = requests.post(
        token_url,
        headers={
            "Authorization": f"Basic {auth}",
            "Content-Type": "application/x-www-form-urlencoded",
        },
        data={"grant_type": "client_credentials", "scope": scope},
    )
    response.raise_for_status()
    return response.json()["access_token"]


print("Helper functions defined")

## Step 3: Get Cognito Configuration

AgentCore Gateway uses JWT authentication. We'll use Cognito to generate tokens.

In [ ]:
# Get Cognito configuration from SSM
try:
    cognito_client_id = get_ssm_parameter("/app/customersupport/agentcore/client_id")
    cognito_pool_id = get_ssm_parameter("/app/customersupport/agentcore/pool_id")
    cognito_discovery_url = get_ssm_parameter("/app/customersupport/agentcore/cognito_discovery_url")
    cognito_token_url = get_ssm_parameter("/app/customersupport/agentcore/cognito_token_url")
    cognito_scope = get_ssm_parameter("/app/customersupport/agentcore/cognito_auth_scope")
    
    print("Cognito configuration loaded from SSM:")
    print(f"  Client ID: {cognito_client_id[:20]}...")
    print(f"  Pool ID: {cognito_pool_id}")
    print(f"  Discovery URL: {cognito_discovery_url}")
except Exception as e:
    print(f"Error loading Cognito config: {e}")
    print("Make sure you've deployed the Cognito stack: make deploy-cognito")

In [ ]:
# Get client secret from Cognito
try:
    client_response = cognito_client.describe_user_pool_client(
        UserPoolId=cognito_pool_id,
        ClientId=cognito_client_id,
    )
    client_secret = client_response["UserPoolClient"]["ClientSecret"]
    print("Client secret retrieved successfully")
except Exception as e:
    print(f"Error getting client secret: {e}")
    client_secret = None

In [ ]:
# Get bearer token for Gateway authentication
if client_secret:
    bearer_token = get_cognito_token(
        cognito_client_id,
        client_secret,
        cognito_token_url,
        cognito_scope,
    )
    print(f"Bearer token obtained: {bearer_token[:50]}...")
else:
    bearer_token = None
    print("No bearer token - Gateway auth will not work")

## Step 4: Create AgentCore Gateway

The Gateway provides a centralized MCP endpoint for tools with JWT-based authentication.

In [ ]:
gateway_name = "workshop-customer-support-gateway"

# Get Gateway IAM role from SSM
gateway_role_arn = get_ssm_parameter("/app/customersupport/agentcore/gateway_iam_role")
print(f"Gateway IAM Role: {gateway_role_arn}")

# Configure JWT authorizer
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [cognito_client_id],
        "discoveryUrl": cognito_discovery_url,
    }
}

try:
    # Create new gateway
    print(f"Creating gateway: {gateway_name}")
    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration=auth_config,
        description="Customer Support Workshop Gateway",
    )
    
    gateway_id = create_response["gatewayId"]
    gateway_url = create_response["gatewayUrl"]
    gateway_arn = create_response["gatewayArn"]
    
    # Store in SSM for later use
    put_ssm_parameter("/app/customersupport/agentcore/gateway_id", gateway_id)
    put_ssm_parameter("/app/customersupport/agentcore/gateway_url", gateway_url)
    put_ssm_parameter("/app/customersupport/agentcore/gateway_arn", gateway_arn)
    
    print(f"Gateway created successfully!")
    print(f"  ID: {gateway_id}")
    print(f"  URL: {gateway_url}")
    
except gateway_client.exceptions.ConflictException:
    # Gateway already exists, get existing
    print(f"Gateway '{gateway_name}' already exists, retrieving...")
    gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = gateway_response["gatewayUrl"]
    gateway_arn = gateway_response["gatewayArn"]
    print(f"Using existing gateway:")
    print(f"  ID: {gateway_id}")
    print(f"  URL: {gateway_url}")
    
except Exception as e:
    print(f"Error creating gateway: {e}")
    gateway_id = None
    gateway_url = None

## Step 5: Add Lambda Function as Gateway Target

We'll add the Lambda function (deployed via CloudFormation) as a Gateway target with tool schema.

In [ ]:
# Tool schema for the Lambda function
tool_schema = [
    {
        "name": "check_warranty_status",
        "description": "Check the warranty status of a product using its serial number",
        "inputSchema": {
            "type": "object",
            "properties": {
                "serial_number": {
                    "type": "string",
                    "description": "The product serial number"
                },
                "customer_email": {
                    "type": "string",
                    "description": "Optional customer email for verification"
                }
            },
            "required": ["serial_number"]
        }
    },
    {
        "name": "web_search",
        "description": "Search the web for updated information",
        "inputSchema": {
            "type": "object",
            "properties": {
                "keywords": {
                    "type": "string",
                    "description": "The search query keywords"
                },
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of results to return"
                }
            },
            "required": ["keywords"]
        }
    }
]

print(f"Tool schema defined with {len(tool_schema)} tools")

In [ ]:
if gateway_id:
    # Get Lambda ARN from SSM
    lambda_arn = get_ssm_parameter("/app/customersupport/agentcore/lambda_arn")
    print(f"Lambda ARN: {lambda_arn}")
    
    # Configure Lambda target
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": lambda_arn,
                "toolSchema": {"inlinePayload": tool_schema},
            }
        }
    }
    
    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]
    
    try:
        # Create gateway target
        target_response = gateway_client.create_gateway_target(
            gatewayIdentifier=gateway_id,
            name="workshop-lambda-tools",
            description="Workshop Lambda tools (warranty, web search)",
            targetConfiguration=lambda_target_config,
            credentialProviderConfigurations=credential_config,
        )
        print(f"Gateway target created: {target_response['targetId']}")
        
    except gateway_client.exceptions.ConflictException:
        print("Gateway target already exists")
        
    except Exception as e:
        print(f"Error creating gateway target: {e}")
else:
    print("Skipping target creation - no gateway")

## Step 6: Set Up MCP Client

The MCP client connects to the Gateway and provides tools to the agent.

In [ ]:
if gateway_url and bearer_token:
    print(f"Gateway Endpoint: {gateway_url}")
    
    # Set up MCP client with authentication
    mcp_client = MCPClient(
        lambda: streamablehttp_client(
            gateway_url,
            headers={"Authorization": f"Bearer {bearer_token}"},
        )
    )
    print("MCP client configured")
else:
    mcp_client = None
    print("MCP client not available - missing gateway URL or token")

## Step 7: Create Agent with Gateway Tools

We combine local tools with MCP tools from the Gateway.

In [ ]:
from utils.agent_config import MODEL_SONNET, OPTIMIZED_SYSTEM_PROMPT
from utils.tools import get_product_info, get_return_policy, get_technical_support

# Initialize the Bedrock model
model = BedrockModel(
    model_id=MODEL_SONNET,
    temperature=0.3,
    region_name=region,
)


def create_gateway_agent(prompt):
    """Create agent with Gateway tools and invoke it."""
    # Local tools
    local_tools = [
        get_product_info,
        get_return_policy,
        get_technical_support,
    ]
    
    if mcp_client:
        # Use MCP client to get Gateway tools
        with mcp_client:
            gateway_tools = mcp_client.list_tools_sync()
            all_tools = local_tools + gateway_tools
            print(f"Tools loaded: {len(local_tools)} local + {len(gateway_tools)} gateway")
            
            agent = Agent(
                model=model,
                tools=all_tools,
                system_prompt=OPTIMIZED_SYSTEM_PROMPT,
            )
            return agent(prompt)
    else:
        # Fallback to local tools only
        print(f"Tools loaded: {len(local_tools)} local (no gateway)")
        agent = Agent(
            model=model,
            tools=local_tools,
            system_prompt=OPTIMIZED_SYSTEM_PROMPT,
        )
        return agent(prompt)


print("Gateway agent function defined")

## Step 8: Test the Gateway Agent

In [ ]:
test_prompts = [
    "What is your return policy for laptops?",
    "Check warranty status for serial number ABC12345678",
    "Tell me about the SmartPhone Pro specifications",
    "My laptop won't turn on, help me troubleshoot",
]

print("=" * 60)
print("Testing Gateway Agent")
print("=" * 60)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\nTest {i}: {prompt}")
    print("-" * 50)
    try:
        response = create_gateway_agent(prompt)
        print(f"Response: {str(response)[:200]}...")
    except Exception as e:
        print(f"Error: {e}")

## Step 9: Deploy to AgentCore Runtime

Now let's deploy the Gateway agent to AgentCore Runtime for production use.

In [ ]:
# Review the v6 agent code
agent_file = Path("agents/v6_gateway.py")
print(agent_file.read_text())

In [ ]:
agent_name = "customer_support_v6_gateway"
agent_file = str(Path("agents/v6_gateway.py").absolute())
requirements_file = str(Path("requirements-for-agentcore.txt").absolute())

print(f"Configuring agent: {agent_name}")
agentcore_runtime.configure(
    entrypoint=agent_file,
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file=requirements_file,
    region=region,
    agent_name=agent_name,
)

In [ ]:
# Modify Dockerfile for Langfuse
import re

dockerfile_path = Path("Dockerfile")
if dockerfile_path.exists():
    content = dockerfile_path.read_text()
    if "opentelemetry-instrument" in content:
        content = re.sub(
            r'CMD \["opentelemetry-instrument", "python", "-m", "([^"]+)"\]',
            r'CMD ["python", "-m", "\1"]',
            content
        )
        dockerfile_path.write_text(content)
        print("Dockerfile modified for Langfuse")
    else:
        print("Dockerfile already configured or using different format")

In [ ]:
env_vars = {
    "LANGFUSE_HOST": os.environ.get("LANGFUSE_HOST"),
    "LANGFUSE_PUBLIC_KEY": os.environ.get("LANGFUSE_PUBLIC_KEY"),
    "LANGFUSE_SECRET_KEY": os.environ.get("LANGFUSE_SECRET_KEY"),
    "GATEWAY_URL": gateway_url or "",
    "GUARDRAIL_ID": os.environ.get("GUARDRAIL_ID", ""),
    "PYTHONUNBUFFERED": "1",
}

print("Deploying to AgentCore Runtime...")
launch_result = agentcore_runtime.launch(env_vars=env_vars, auto_update_on_conflict=True)
agent_arn = launch_result.agent_arn
print(f"Agent deployed: {agent_arn}")

## Step 10: Test Deployed Agent

In [ ]:
data_client = boto3.client("bedrock-agentcore", region_name=region)


def invoke_agent(prompt):
    """Invoke the deployed agent via AgentCore API."""
    response = data_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        runtimeSessionId=str(uuid.uuid4()),
        payload=json.dumps({"prompt": prompt}).encode(),
    )
    return json.loads(response["response"].read().decode("utf-8"))

In [ ]:
# Import Langfuse metrics helper
from utils.langfuse_metrics import (
    get_latest_trace_metrics,
    print_metrics,
    clear_metrics,
    collect_metric,
    print_metrics_table,
    get_collected_metrics
)

# Clear any previously collected metrics
clear_metrics()
print("Metrics helper ready")

In [ ]:
# Standard test prompts - same across all notebooks for consistent comparison
TEST_PROMPTS = [
    # Single tool: get_return_policy
    ("Return Policy", "What is your return policy for laptops?"),

    # Single tool: get_product_info
    ("Product Info", "Tell me about your smartphone options"),

    # Single tool: get_technical_support (Bedrock KB)
    ("Technical Support", "My laptop won't turn on, can you help me troubleshoot?"),

    # Multi-tool: get_product_info + get_return_policy
    ("Multi-part Question", "I want to buy a laptop. What are the specs and what's the return policy?"),

    # No tool: General greeting
    ("General Question", "Hello! What can you help me with today?"),
]

# Run all tests and collect metrics
print("=" * 60)
print("Testing Deployed Gateway Agent")
print("=" * 60)

for test_name, prompt in TEST_PROMPTS:
    print("\n" + "=" * 60)
    print(f"Test: {test_name}")
    print("=" * 60)

    response = invoke_agent(prompt)
    print(f"Response: {response}")

    # Fetch and collect metrics
    metrics = get_latest_trace_metrics(
        agent_name="customer-support-v6-gateway",
        wait_seconds=5,
        max_retries=5,
        timeout_seconds=120,
    )
    print_metrics(metrics, test_name)
    collect_metric(metrics, test_name)

In [ ]:
# Print summary table and comparison vs baseline
print_metrics_table()

# Baseline metrics from notebook 01
BASELINE_AVG_INPUT_TOKENS = 4251
BASELINE_AVG_LATENCY = 8.0

# Calculate improvements
collected = get_collected_metrics()
if collected:
    valid_metrics = [m for m in collected if "error" not in m]
    if valid_metrics:
        avg_input = sum(m.get('input_tokens', 0) for m in valid_metrics) / len(valid_metrics)
        avg_latency = sum(m.get('latency_seconds', 0) or 0 for m in valid_metrics) / len(valid_metrics)

        token_reduction = ((BASELINE_AVG_INPUT_TOKENS - avg_input) / BASELINE_AVG_INPUT_TOKENS) * 100
        latency_change = ((BASELINE_AVG_LATENCY - avg_latency) / BASELINE_AVG_LATENCY) * 100

        print("\n" + "=" * 60)
        print("           COMPARISON VS BASELINE (v1)")
        print("=" * 60)
        print(f"  Avg Input Tokens:  {avg_input:,.0f} (Baseline: {BASELINE_AVG_INPUT_TOKENS:,})")
        print(f"  Token Reduction:   {token_reduction:+.1f}%")
        print(f"  Avg Latency:       {avg_latency:.2f}s (Baseline: {BASELINE_AVG_LATENCY:.2f}s)")
        print(f"  Latency Change:    {latency_change:+.1f}%")
        print("=" * 60)
        print("\n📝 Note: Gateway provides semantic tool search - loads only relevant tools per query")
        print("   This reduces context size and improves efficiency for agents with many tools.")

## Summary

In this notebook, we integrated AgentCore Gateway:

1. **Created Gateway** with JWT authentication via Cognito
2. **Added Lambda target** with warranty and web search tools
3. **Connected via MCP** using the Strands MCPClient
4. **Deployed to Runtime** for production use

**Benefits:**
- **Centralized tools**: Multiple agents share the same tools
- **Secure access**: JWT-based authentication
- **Easy updates**: Change tools in one place

**Next Steps:** In the final notebook, we'll run comprehensive evaluations across all versions.

**Next notebook:** [07-evaluations.ipynb](./07-evaluations.ipynb)

## Cleanup (Optional)

In [ ]:
# Uncomment to delete the gateway
# if gateway_id:
#     gateway_client.delete_gateway(gatewayIdentifier=gateway_id)
#     print(f"Deleted gateway: {gateway_id}")